In [10]:
import sp_model 
from util import to_range, DATA_PATH, FIG_PATH, OptimizationMethod

w = 0.2


# %%
# 有一個趨勢是幾乎每個點都會開立一個rdc, cs，很像沒有不開的。
m, obj1, obj2 = sp_model.solve(1, OptimizationMethod.WEIGHTED_SUM)
obj1_star = obj1
m, obj1, obj2 = sp_model.solve(0, OptimizationMethod.WEIGHTED_SUM)
obj2_star = obj2
objstars = [obj1_star, obj2_star]
spm, o1, o2 = sp_model.solve(w,
                OptimizationMethod.LP_METRIC,
                objstars)

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 20 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 4636 rows, 4452 columns and 13080 nonzeros
Model fingerprint: 0xa75f41df
Model has 666 quadratic constraints
Model has 12 general constraints
Variable types: 4422 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+13]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [3e-01, 1e+00]
  Objective range  [2e-03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
  QRHS range       [7e+00, 2e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---

In [11]:
# get supplier names 
import pandas as pd 
PATH_PREFIX = 'MoDRL_'
df_supplier = pd.read_csv(DATA_PATH 
+ f'/{PATH_PREFIX}supplier.csv')
suppliers = df_supplier['Suppliers'].values 

df_distance = pd.read_csv(DATA_PATH + f'/{PATH_PREFIX}distance.csv')
AA = df_distance.columns.values
print(suppliers)
print(AA)
Commodities = ['Water', 'Food', 'Shelter']

['Sari' 'Qazvin' 'Tehran' 'Arak' 'Isfahan']
['Gorgan' 'Semnan' 'Sari' 'Rasht' 'Qazvin' 'Karaj' 'Tehran' 'Varamin'
 'Robatkarim' 'Islamshahr' 'Shahriar' 'Gom' 'Arak' 'Isfahan' 'Kashan']


### $\alpha_j, \beta_j$ Build RDC/CS or not 

In [12]:
from sp_model import SET, DELTA, PARAMETER
alpha, beta = [], [] 
for j in to_range(SET['J']):
    alpha_j = spm.getVarByName(f'alpha[{j}]').x
    beta_j = spm.getVarByName(f'beta[{j}]').x
    # alpha_i: build RDC or not 
    # beta_i: build CS or not
    alpha.append(alpha_j)
    beta.append(beta_j)
    # print(AA[i], alpha_i, beta_i)
build_df = pd.DataFrame({'RDC': alpha, 'CS': beta}, index=AA)
build_df = build_df.transpose() 
build_df

,Gorgan,Semnan,Sari,Rasht,Qazvin,Karaj,Tehran,Varamin,Robatkarim,Islamshahr,Shahriar,Gom,Arak,Isfahan,Kashan
RDC,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
CS,-0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [13]:
print(sum(alpha))
sum(beta)

10.0


5.0

In [5]:
i, j, k, k_prime, s, c = [len(idx) for idx in SET.values()]
import numpy as np 
Q = np.zeros((i, j, c))
X = np.zeros((i, j, c, s))
Y = np.zeros((j, k, c, s))

### $Q_{ijc}, X_{ijcs}$ transported commodities to RDC/CS

In [14]:
## get Q_ijc: Sup to RDC/Cs in preparedness phase 
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            Q[i, j, c] = spm.getVarByName(f'Q[{i},{j},{c}]').x
# get X_ijcs: Sup to RDC/Cs in response phase 
print('======= RDC =======')
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            s_vec = np.zeros(len(SET['S']))
            for s in to_range(SET['S']):
                X[i, j, c, s] = spm.getVarByName(f'X[{i},{j},{c},{s}]').x
                s_vec[s] = X[i, j, c, s] 
            if np.sum(s_vec) != 0 and alpha[j]: 
                print(f'{suppliers[i]} => {AA[j]}', Commodities[c], s_vec)

======= RDC =======


In [7]:
print(f'======= CS =======')
for i in to_range(SET['I']):
    for j in to_range(SET['J']):
        for c in to_range(SET['C']):
            s_vec = np.zeros(len(SET['S']))
            for s in to_range(SET['S']):
                X[i, j, c, s] = spm.getVarByName(f'X[{i},{j},{c},{s}]').x
                s_vec[s] = X[i, j, c, s] 
            if np.sum(s_vec) != 0 and beta[j]: 
                print(f'{suppliers[i]} => {AA[j]}', Commodities[c], s_vec)


======= CS =======
Sari => Sari Water [  0. 117. 117. 117.]
Sari => Sari Food [117. 117. 117. 117.]
Sari => Sari Shelter [ 0.  0. 39. 39.]
Qazvin => Tehran Food [16.6  0.   0.   0. ]
Qazvin => Shahriar Water [  0. 117. 117. 117.]
Qazvin => Shahriar Food [100.4 117.  117.  117. ]
Qazvin => Shahriar Shelter [ 0.  0. 39. 39.]
Tehran => Tehran Water [55.  0.  0.  0.]
Tehran => Tehran Food [132.6   0.    0.    0. ]
Tehran => Islamshahr Water [  0.  132.6 132.6 132.6]
Tehran => Islamshahr Food [  0.  132.6 132.6 132.6]
Tehran => Islamshahr Shelter [ 0.  44.2 44.2 44.2]
Arak => Tehran Food [117.   0.   0.   0.]
Arak => Islamshahr Water [  0. 117. 117. 117.]
Arak => Islamshahr Food [  0. 117. 117. 117.]
Arak => Islamshahr Shelter [ 0. 39. 39. 39.]
Isfahan => Isfahan Water [  0.   0. 117. 117.]
Isfahan => Isfahan Food [  0. 117.   0. 117.]
Isfahan => Isfahan Shelter [ 0.  0. 39.  0.]
Isfahan => Kashan Food [  0.   0. 117.   0.]


### $Y_{jkcs}$ transported commodities to AA

In [15]:
    
def isNotZero(x):
    return abs(x) > 1e-6
for s in to_range(SET['S']):
    print('s=', s)
    for j in to_range(SET['J']):
        for k in to_range(SET['K']):
            for c in to_range(SET['C']):
                Y[j, k, c, s] = spm.getVarByName(f'Y[{j},{k},{c},{s}]').x
                identity = 'RDC' if alpha[j] == 1 else ''
                identity = 'CS' if beta[j] == 1 else '' 
                if isNotZero(Y[j, k, c, s]):
                    # J and K share the same node set 
                    print(f'{identity} {AA[j]} => {AA[k]}', Commodities[c], Y[j, k, c, s])

s= 0
CS Tehran => Sari Food 55.0
s= 1
CS Sari => Tehran Food 117.0
CS Islamshahr => Tehran Water 249.6
CS Islamshahr => Tehran Food 249.6
CS Islamshahr => Tehran Shelter 83.2
CS Shahriar => Tehran Water 117.0
CS Shahriar => Tehran Food 117.0
CS Isfahan => Tehran Food 117.0
s= 2
CS Sari => Tehran Water 117.0
CS Sari => Tehran Food 117.0
 Karaj => Arak Food 75.0
CS Islamshahr => Tehran Water 249.6
CS Islamshahr => Tehran Food 249.6
CS Islamshahr => Tehran Shelter 83.2
CS Shahriar => Tehran Water 117.0
CS Shahriar => Tehran Food 117.0
CS Shahriar => Tehran Shelter 39.0
CS Isfahan => Tehran Water 117.0
CS Isfahan => Tehran Food 117.0
s= 3
CS Sari => Tehran Water 117.0
CS Sari => Tehran Food 117.0
CS Islamshahr => Tehran Water 249.6
CS Islamshahr => Tehran Food 249.6
CS Islamshahr => Tehran Shelter 83.2
CS Shahriar => Tehran Water 117.0
CS Shahriar => Tehran Food 117.0
CS Isfahan => Tehran Food 117.0


### $I_{kcs}$: the inventory held at AA k for commod c under s 
### $b_{kcs}$: the shortage held at AA k for commod c under s  

In [9]:
k, c, s = len(SET['K']), len(SET['C']), len(SET['S'])
I = np.zeros((k, c, s))
b = np.zeros((k, c, s))

for s in to_range(SET['S']):
    print('s=', s)
    for k in to_range(SET['K']):
        for c in to_range(SET['C']):
            
            I[k, c, s] = spm.getVarByName(f'I[{k},{c},{s}]').x
            b[k, c, s] = spm.getVarByName(f'b[{k},{c},{s}]').x
            
            if isNotZero(I[k, c, s]):
                print(f'inventory: AA {k}', Commodities[c], I[k, c, s])
            if isNotZero(b[k, c, s]):
                print(f'shortage: AA {k}', Commodities[c], b[k, c, s])
                

s= 0
shortage: AA 0 Water 319.0
shortage: AA 0 Food 312.8
shortage: AA 0 Shelter 106.0
shortage: AA 1 Water 34.0
shortage: AA 1 Food 34.0
shortage: AA 1 Shelter 11.0
shortage: AA 2 Water 524.0
shortage: AA 2 Food 312.8
shortage: AA 2 Shelter 193.0
shortage: AA 3 Water 524.0
shortage: AA 3 Food 312.8
shortage: AA 3 Shelter 175.0
shortage: AA 4 Water 68.0
shortage: AA 4 Food 68.0
shortage: AA 4 Shelter 23.0
s= 1
shortage: AA 0 Water 159.0
shortage: AA 0 Food 159.0
shortage: AA 0 Shelter 53.0
shortage: AA 1 Water 69.0
shortage: AA 1 Food 69.0
shortage: AA 1 Shelter 23.0
shortage: AA 2 Water 521.0
shortage: AA 2 Food 521.0
shortage: AA 2 Shelter 174.0
inventory: AA 3 Water 505.4000000000001
shortage: AA 3 Water 934.4000000000001
shortage: AA 3 Food 429.0
shortage: AA 3 Shelter 143.0
shortage: AA 4 Water 169.0
shortage: AA 4 Food 169.0
shortage: AA 4 Shelter 56.0
inventory: AA 5 Water 641.4000000000001
shortage: AA 5 Water 934.4000000000001
shortage: AA 5 Food 293.0
shortage: AA 5 Shelter 9